In [2]:
import tensorflow as tf

In [3]:
net = tf.keras.models.Sequential([tf.keras.layers.Dense(256,activation = tf.nn.relu),
                                  tf.keras.layers.Dense(16),
                                 ])

X = tf.random.uniform((2,20))
net(X)

<tf.Tensor: shape=(2, 16), dtype=float32, numpy=
array([[ 0.03793888,  0.03837431,  0.07881299, -0.01210025, -0.23910588,
         0.069548  , -0.17152178, -0.3544281 ,  0.11446775,  0.05249958,
        -0.15397947,  0.06283358,  0.09527606, -0.03975724, -0.33828694,
         0.16966903],
       [ 0.04954688,  0.03473268,  0.01283243,  0.18453446, -0.24783552,
         0.20232323, -0.2460535 , -0.39256862,  0.36996096,  0.08043034,
        -0.09084583, -0.00157549,  0.17253639, -0.12066321, -0.0892628 ,
         0.1797708 ]], dtype=float32)>

In [6]:
class MLP(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(256,activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(10)
    
    def call(self,X):
        return self.out(self.hidden((X)))

In [9]:
net = MLP()
net(X)


<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.5648432 , 0.5074893 , 0.83366716, 0.86529446, 0.36031997,
        0.5900333 , 0.4224981 , 0.32877505, 0.809008  , 0.32848155,
        0.7628732 , 0.29490972, 0.468346  , 0.3231895 , 0.09616756,
        0.99117815, 0.53065145, 0.24529135, 0.6122583 , 0.34063423],
       [0.5407821 , 0.00309324, 0.6825639 , 0.6181811 , 0.2304883 ,
        0.24587512, 0.78691196, 0.92343867, 0.23041868, 0.3842492 ,
        0.6716305 , 0.48350096, 0.09825265, 0.1650809 , 0.18695128,
        0.1655811 , 0.33515513, 0.84262824, 0.18564045, 0.45268   ]],
      dtype=float32)>

In [17]:
class MySeq(tf.keras.Model):
    def __init__(self,*args):
        super().__init__()
        self.modules = []
        for block in args:
            self.modules.append(block)
        
    
    def call(self,X):
        for module in self.modules:
            X = module(X)
        
        return X

In [18]:
net = MySeq(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.5202818 ,  0.4089492 , -0.1552027 , -0.02211266, -0.20047441,
         0.02488761, -0.3818569 , -0.26887792, -0.27240628,  0.13486868],
       [-0.25035575,  0.32752466, -0.16890489,  0.06006361, -0.31214896,
        -0.02258125, -0.21423237, -0.21915725, -0.18722586,  0.08279666]],
      dtype=float32)>

In [24]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(20,activation = tf.nn.relu)
    
    def call(self,inputs):
        X = self.flatten(inputs)
        X = tf.nn.relu(tf.matmul(X,self.rand_weight)+1)
        X = self.dense(X)
        
        while tf.reduce_sum(tf.math.abs(X))>1:
            X/=2
        
        return tf.reduce_sum(X)
    

In [25]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7846392>

In [31]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64,activation=tf.nn.sigmoid))
        self.net.add(tf.keras.layers.Dense(32,activation=tf.nn.sigmoid))
        self.dense = tf.keras.layers.Dense(8)
    
    def call(self,inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20,activation=tf.nn.relu))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7488892>

In [44]:
class ParallelNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net1 =tf.keras.Sequential()
        self.net1.add(tf.keras.layers.Dense(64,activation=tf.nn.relu))
        self.net1.add(tf.keras.layers.Dense(32,activation=tf.nn.relu))
        self.net2 =tf.keras.Sequential()
        self.net2.add(tf.keras.layers.Dense(32,activation=tf.nn.relu))
        self.net2.add(tf.keras.layers.Dense(16,activation=tf.nn.relu))
    
    def call(self,inputs):
        return tf.concat([self.net1(inputs),self.net2(inputs)],axis=1)
        

In [45]:
test = ParallelNet()
test(X)

<tf.Tensor: shape=(2, 48), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.01999505, 0.        ,
        0.        , 0.25348017, 0.45639056, 0.3420291 , 0.04271244,
        0.7868906 , 0.02009377, 0.16264716, 0.        , 0.        ,
        0.        , 0.17544225, 0.        , 0.28852427, 0.05138777,
        0.        , 0.37894076, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.1694813 , 0.02351716,
        0.33409226, 0.        , 0.        , 0.0605364 , 0.        ,
        0.        , 0.        , 0.6479514 , 0.47136533, 0.        ,
        0.38354614, 0.        , 0.0941817 , 0.        , 0.        ,
        0.0858431 , 0.33914393, 0.        ],
       [0.        , 0.        , 0.        , 0.18149623, 0.16832739,
        0.        , 0.        , 0.16856319, 0.09710114, 0.08903161,
        0.44052768, 0.1532836 , 0.01553615, 0.        , 0.        ,
        0.        , 0.08770853, 0.1512183 , 0.15587816, 0.        ,
        0.        , 0.